In [ ]:
import pandas as pd

try:
    # Attempt to read with comma as delimiter
    df = pd.read_csv('/content/mergedmessagesInfluencerFinal.csv', encoding='latin1', delimiter=',')
    print("File read successfully with comma as delimiter!")
except Exception as e:
    print("Error with comma:", e)

try:
    # Attempt to read with semicolon as delimiter
    df = pd.read_csv('/content/mergedmessagesInfluencerFinal.csv', encoding='latin1', delimiter=';')
    print("File read successfully with semicolon as delimiter!")
except Exception as e:
    print("Error with semicolon:", e)

try:
    # Attempt to read with tab as delimiter
    df = pd.read_csv('/content/mergedmessagesInfluencerFinal.csv', encoding='latin1', delimiter='\t')
    print("File read successfully with tab as delimiter!")
except Exception as e:
    print("Error with tab:", e)


Error with comma: Error tokenizing data. C error: Expected 1 fields in line 3, saw 9

File read successfully with semicolon as delimiter!
File read successfully with tab as delimiter!


# Die Dateien sind zu groß und deshalb stürzt Bertopic ab. mergedmessages-Dateien werden in x Teile aufgesplittet und Topic Modeling  anschließend für jeden Part angewandt.

Datei in Teile aufteilen

In [ ]:
import pandas as pd

# Pfad zur großen CSV-Datei
csv_file = '/content/mergedmessagesLokalFinal.csv'

# CSV-Datei in ein DataFrame einlesen
df = pd.read_csv(csv_file, encoding= 'latin1',sep=';', on_bad_lines='skip')

# Anzahl der Zeilen in der CSV-Datei
total_rows = len(df)

# Berechne die Größe jedes Teilstücks (ungefähr ein Zehntel der gesamten Zeilen)
chunk_size = total_rows // 10

# Aufteilen in 10 Teile
chunks = [df[i:i+chunk_size] for i in range(0, total_rows, chunk_size)]

# Sicherstellen, dass alle Zeilen verteilt sind, falls die Gesamtanzahl nicht genau durch 10 teilbar ist
if len(chunks) > 10:
    chunks[-2] = pd.concat([chunks[-2], chunks[-1]], ignore_index=True)
    chunks = chunks[:-1]

# Die Teilstücke in separate CSV-Dateien speichern
for i, chunk in enumerate(chunks):
    chunk.to_csv(f'messages_lokal_teil_{i+1}.csv', index=False)

print("Dateien erfolgreich aufgeteilt und gespeichert.")


Dateien erfolgreich aufgeteilt und gespeichert.


# Preprocessing jedes Parts

In [ ]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup

from google.colab import drive
drive.mount('/content/drive')

# Funktion zur Textbereinigung
def clean_text(text):
    if isinstance(text, str):
        # Entferne HTML-Tags
        text = BeautifulSoup(text, "html.parser").get_text()

        # Entferne URLs
        text = re.sub(r'http\S+|www.\S+', '', text)

        # Entferne Sonderzeichen, Emoticons etc.
        text = re.sub(
            u"[" +
            u"\U0001F600-\U0001F64F"  # Emoticons
            u"\U0001F300-\U0001F5FF"  # Symbole & Piktogramme
            u"\U0001F680-\U0001F6FF"  # Transport & Karte
            u"\U0001F1E0-\U0001F1FF"  # Flaggen
            u"\U00002702-\U000027B0"  # Dingbats
            u"\U000024C2-\U0001F251"  # Verschiedene Symbole
            r"\|®"  # Entferne | und ®
            "]+", '', text)

        # Entferne führende und nachfolgende Leerzeichen
        text = text.strip()
        return text.lower()  # Konvertiere zu Kleinbuchstaben
    else:
        return ""  # Gib leere Zeichenkette zurück, falls kein Text

# Bereinigung und Verarbeitung für jeden Teil
for i in range(1, 11):
    try:
        # Lese den jeweiligen Teil der Datei ein
        df = pd.read_csv(f'/content/messages_lokal_teil_{i}.csv', encoding='utf-8', delimiter=',', on_bad_lines='skip', header=0)
        print(f"Teil {i} erfolgreich gelesen!")

        # Spaltennamen bereinigen
        df.columns = df.columns.str.strip()

        # Prüfen Sie die Namen der Spalten, um sicherzustellen, dass 'Forwarded Message Text' korrekt ist
        if 'Message Text' not in df.columns or 'Forwarded Message Text' not in df.columns:
            print(f"Fehler: Erwartete Spalten nicht gefunden in Teil {i}")
            continue

        # Wende die Reinigungsfunktion auf die Spalte 'Message Text' an
        df['Message Text'] = df['Message Text'].apply(clean_text)

        # Erstellen Sie eine neue Spalte 'Is_Forwarded', die 'Ja' enthält, wenn 'Forwarded Message Text' signifikanten Inhalt hat, sonst 'Nein'
        df['Is_Forwarded'] = np.where(df['Forwarded Message Text'].notna() & df['Forwarded Message Text'].str.strip().apply(bool), 'Ja', 'Nein')

        # Behalte nur die Spalten 'Message Text' und 'Is_Forwarded'
        final_df = df[['Message Text', 'Is_Forwarded']]

        # Entferne Zeilen, bei denen 'Message Text' NaN oder leer ist
        final_df.dropna(subset=['Message Text'], inplace=True)
        final_df = final_df[final_df['Message Text'].str.strip() != '']

        # Anzeigen der ersten Zeilen des neuen DataFrame zur Überprüfung
        print(f"Erste Zeilen des bereinigten DataFrame für Teil {i}:")
        print(final_df.head())

        # Speichern der bereinigten Teile für das Topic Modeling
        final_df.to_csv(f'/content/drive/My Drive/MasterArbeitDatenLokal/messages_lokal_teil_{i}_cleaned_forwarded.csv', index=False)
        print(f"Teil {i} erfolgreich bereinigt und gespeichert!")

    except Exception as e:
        print(f"Fehler beim Verarbeiten von Teil {i}: {e}")
        continue


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Teil 1 erfolgreich gelesen!


<ipython-input-5-96f213a40811>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
<ipython-input-5-96f213a40811>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.dropna(subset=['Message Text'], inplace=True)


Erste Zeilen des bereinigten DataFrame für Teil 1:
                                        Message Text Is_Forwarded
0  astrazeneca-impfung kann allergien auslã¶senâ...         Nein
1                                             â¤ï¸         Nein
2  der bericht einer ãrztin. unbedingt anhã¶ren ...         Nein
3               20.03.21: captain future in potsdam!         Nein
4  hi und warum in potsdam und nicht in berlin......         Nein
Teil 1 erfolgreich bereinigt und gespeichert!
Teil 2 erfolgreich gelesen!


<ipython-input-5-96f213a40811>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
<ipython-input-5-96f213a40811>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.dropna(subset=['Message Text'], inplace=True)


Erste Zeilen des bereinigten DataFrame für Teil 2:
                                        Message Text Is_Forwarded
0  ð¨ð­corona im altersheim in der schweiz - ...           Ja
1  #pdf merkel und spahn planen das grosse organh...           Ja
2                                        @unzensiert           Ja
3  keine panik die menschen werden es nicht mitbe...         Nein
4                                        @unzensiert           Ja
Teil 2 erfolgreich bereinigt und gespeichert!
Teil 3 erfolgreich gelesen!


...

Erste Zeilen des bereinigten DataFrame für Teil 8:
                                        Message Text Is_Forwarded
0  das gab es alles schon mal . die brd als recht...         Nein
2                                            a.c.a.b           Ja
3  du sagst es: "eher trã¤ge". dem regime rennt a...         Nein
4  ð³ny-post verwendet bilder einer gasleck-kat...           Ja
8  so sieht das aus. wir haben derzeit 7% geimpft...         Nein
Teil 8 erfolgreich bereinigt und gespeichert!
Teil 9 erfolgreich gelesen!


<ipython-input-5-96f213a40811>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
<ipython-input-5-96f213a40811>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.dropna(subset=['Message Text'], inplace=True)


Erste Zeilen des bereinigten DataFrame für Teil 9:
                                        Message Text Is_Forwarded
0  autokorso berlin, wahnsinn. wir werden erfahre...         Nein
2  autokorso gegen #corona-maãnahmen fã¤hrt mit ...           Ja
3  bock auf ne bauernsolidaritã¤tsdemo heute auf ...           Ja
4  â ï¸berlinâ ï¸ angemeldete grossdemonstrat...           Ja
5  bock auf ne bauernsolidaritã¤tsdemo heute auf ...         Nein
Teil 9 erfolgreich bereinigt und gespeichert!
Teil 10 erfolgreich gelesen!


<ipython-input-5-96f213a40811>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
<ipython-input-5-96f213a40811>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.dropna(subset=['Message Text'], inplace=True)


Erste Zeilen des bereinigten DataFrame für Teil 10:
                                        Message Text Is_Forwarded
0                                               ðª         Nein
1              sehr gut, hab mich gerade abgemeldet.         Nein
2  werner, vielen dank fã¼r deinen einsatz! ich w...         Nein
3                                               ð¢         Nein
4                     ich kann am 3.10. leider nicht         Nein
Teil 10 erfolgreich bereinigt und gespeichert!


In [ ]:
import os
os.environ['JAX_NUM_THREADS'] = '1'

# Durchführen des Topic Modelings mit BERTopic

In [ ]:
!pip install -q bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.8 MB/s eta 0:00:00


In [ ]:
%%capture
!pip install --upgrade bertopic umap-learn hdbscan jax jaxlib

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

STOPWORDS = stopwords.words('german')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=STOPWORDS)

In [ ]:
from joblib import parallel_backend
from bertopic import BERTopic
import pandas as pd
from google.colab import drive

# Google Drive mounten
drive.mount('/content/drive')

# Funktion zum Speichern der Top-Wörter und Themeninformationen
def save_topic_words(topic_model, file_name, num_words=10):
    # Hole die Top-Wörter und die Informationen zu den Themen
    topics = topic_model.get_topics()
    topic_info = topic_model.get_topic_info()

    # Erstelle ein Dictionary mit den Top-Wörtern und den jeweiligen Counts
    topic_words = {f"Topic {topic_num} (Count: {int(topic_info[topic_info['Topic'] == topic_num]['Count'])})":
                   [word for word, _ in words[:num_words]] for topic_num, words in topics.items()}

    # Wandle das Dictionary in einen DataFrame um
    df_topics = pd.DataFrame.from_dict(topic_words, orient='index')

    # Speichere den DataFrame als CSV-Datei
    df_topics.to_csv(file_name, index=True)


# Topic Modeling für jeden Teil durchführen
for i in range(1, 11):
    try:
        # Lese den bereinigten Teil der Datei ein
        final_df = pd.read_csv(f'/content/messages_influencer_teil_{i}_cleaned.csv')

        # Bereinige die 'Message Text' Spalte, konvertiere in Strings und ersetze NaN durch leere Strings
        final_df['Message Text'] = final_df['Message Text'].astype(str).fillna('')

        # Extrahiere die Nachrichten in eine Liste und entferne leere Einträge
        docs = [doc for doc in final_df['Message Text'].tolist() if isinstance(doc, str) and doc.strip() != '']

        # Verwende parallel_backend für die parallele Verarbeitung
        with parallel_backend('loky'):
            topic_model = BERTopic(language="multilingual", calculate_probabilities=True, verbose=True, vectorizer_model=vectorizer_model)
            topics, probs = topic_model.fit_transform(docs)

        # Themenmodellierungsergebnisse anzeigen
        print(f"Teil {i}: Themenmodellierung abgeschlossen!")
        topic_info = topic_model.get_topic_info()
        print(topic_info)

        # Speichere die Top-Wörter jedes Themas in Google Drive
        save_topic_words(topic_model, f'/content/drive/MyDrive/MasterArbeitDaten/Topic Modeling/topic_model_part_{i}_words.csv')

        # Speichere das Modell in Google Drive
        topic_model.save(f'/content/drive/MyDrive/MasterArbeitDaten/Topic Modeling/topic_model_part_{i}.bertopic')
        print(f"Modell für Teil {i} gespeichert.")

        # Speichern der Wahrscheinlichkeiten der Themen pro Dokument in Google Drive
        df_probs = pd.DataFrame(probs)
        df_probs.to_csv(f'/content/drive/MyDrive/MasterArbeitDaten/Topic Modeling/topic_probabilities_part_{i}.csv', index=False)

    except Exception as e:
        print(f"Fehler bei der Themenmodellierung für Teil {i}: {e}")


2024-09-17 12:13:35,118 - BERTopic - Embedding - Transforming documents to embeddings.


Mounted at /content/drive


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1405 [00:00<?, ?it/s]

2024-09-17 14:47:11,545 - BERTopic - Embedding - Completed ✓
2024-09-17 14:47:11,547 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-09-17 14:47:36,655 - BERTopic - Dimensionality - Completed ✓
2024-09-17 14:47:36,657 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-09-17 15:01:01,659 - BERTopic - Cluster - Completed ✓
2024-09-17 15:01:01,676 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-09-17 15:01:07,312 - BERTopic - Representation - Completed ✓


Teil 10: Themenmodellierung abgeschlossen!
     Topic  Count                                               Name  \
0       -1  19693                            -1_mehr_menschen_ja_mal   
1        0   1389                             0_gut_dubistfrei_ja_oh   
2        1    809           1_impfstoff_impfung_impfstoffe_impfungen   
3        2    770                              2_janich_ja_sagt_weiß   
4        3    767  3_gruppe_libertaerediskussion herzlich_aufmerk...   
..     ...    ...                                                ...   
488    487     10                       487_tirol_montgomery_ghz_led   
489    488     10    488_tik_tik tik_putsch_kerbholz unbeschriebenes   
490    489     10                  489_hiv_ja hiv_aids_überträgst ja   
491    490     10                       490_06_00 live_06 2020_06 06   
492    491     10  491_03 april_stuttgart 03_pressemitteilung que...   

                                        Representation  \
0    [mehr, menschen, ja, mal, cor

2024-09-17 15:01:10,881 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


Modell für Teil 10 gespeichert.


# Analyse der Topic Modeling Ergebnisse

In [ ]:
import pandas as pd

# Lade die Top-Wörter für jedes Thema in den 10 Teilen
topic_words_list = []
for i in range(1, 11):
    df = pd.read_csv(f'/content/drive/MyDrive/MasterArbeitDatenLokal/Topic Modeling/topic_model_part_{i}_words.csv')
    topic_words_list.append(df)

# Zeige die Top-Wörter für das erste Thema (Topic 0) in jedem Teil
for i in range(10):
    print(f"Top-Wörter für Topic  in Teil {i+1}:")
    print(topic_words_list[i].loc[9].dropna().tolist())  # Zeige die Top-Wörter für Topic 0
    print()


In [ ]:
import pandas as pd

# Beispiel: Lade die CSV-Dateien der Top-Wörter für alle 10 Teile
topic_words_list = []
for i in range(1, 11):
    df = pd.read_csv(f'/content/topic_model_part_{i}_words.csv')
    topic_words_list.append(df)

# Vergleich der Top-Wörter für Topic 0 über alle Teile hinweg
for i in range(10):
    print(f"Teil {i+1} - Top-Wörter für Topic :")
    print(topic_words_list[i].loc[0].values)  # Zeige die Top-Wörter für Topic 0-9
    print()


Top-Wörter für Teil 1:
  Unnamed: 0          0                 1          2               3  \
0   Topic -1       mehr          menschen     corona             mal   
1    Topic 0     berlin          berliner       demo      querdenken   
2    Topic 1    polizei        polizisten   polizist   demonstranten   
3    Topic 2  impfstoff           impfung  impfungen      impfstoffe   
4    Topic 3     corona  corona maßnahmen  maßnahmen  corona politik   

                       4             5          6                 7  \
0                  schon            ja      immer             bitte   
1            demo berlin  grüße berlin         10             steht   
2                 gewalt         wurde       demo     polizeigewalt   
3         nebenwirkungen        impfen     pfizer             covid   
4  verfassungsbeschwerde       bergamo  ausschuss  corona ausschuss   

                   8             9  
0              wurde          geht  
1              grüße  berlin steht  
2     

In [ ]:
!pip install bertopic --upgrade
from bertopic import BERTopic
import os

# Liste für alle Themen aus den 10 Teilen
all_topics = {}

# Basisverzeichnis (Anpassen, falls nötig)
base_dir = "/content/drive/MyDrive/MasterArbeitDaten/Topic Modeling/"

# Schleife über alle 10 Teile und lade sie
for i in range(1,11):
    model_path = os.path.join(base_dir, f"topic_model_part_{i}.bertopic")

    # Prüfe, ob die Datei existiert, bevor sie geladen wird
    if os.path.exists(model_path):
        print(f"Lade Modell: {model_path}")
        topic_model = BERTopic.load(model_path)

        # Hol die Themen und füge sie der all_topics Liste hinzu
        topics = topic_model.get_topics()

        # Kombiniere die Themen aus den Teilen
        all_topics.update(topics)
    else:
        print(f"Modell nicht gefunden: {model_path}")

# Jetzt hast du alle Themen aus allen Modellen in all_topics gesammelt.
# Zeige nur die Top-Wörter für die ersten 20 Themen
for topic_id in range(51):
    if topic_id in all_topics:
        print(f"Thema {topic_id}:")
        for word, weight in all_topics[topic_id]:
            print(f"{word} (Gewichtung: {weight})")
        print("\n")
    else:
        print(f"Thema {topic_id} ist nicht vorhanden.\n")



Lade Modell: /content/drive/MyDrive/MasterArbeitDaten/Topic Modeling/topic_model_part_1.bertopic
Lade Modell: /content/drive/MyDrive/MasterArbeitDaten/Topic Modeling/topic_model_part_2.bertopic
Lade Modell: /content/drive/MyDrive/MasterArbeitDaten/Topic Modeling/topic_model_part_3.bertopic
Lade Modell: /content/drive/MyDrive/MasterArbeitDaten/Topic Modeling/topic_model_part_4.bertopic
Lade Modell: /content/drive/MyDrive/MasterArbeitDaten/Topic Modeling/topic_model_part_5.bertopic
Lade Modell: /content/drive/MyDrive/MasterArbeitDaten/Topic Modeling/topic_model_part_6.bertopic
Lade Modell: /content/drive/MyDrive/MasterArbeitDaten/Topic Modeling/topic_model_part_7.bertopic
Lade Modell: /content/drive/MyDrive/MasterArbeitDaten/Topic Modeling/topic_model_part_8.bertopic
Lade Modell: /content/drive/MyDrive/MasterArbeitDaten/Topic Modeling/topic_model_part_9.bertopic
Lade Modell: /content/drive/MyDrive/MasterArbeitDaten/Topic Modeling/topic_model_part_10.bertopic
Thema 0:
gut (Gewichtung: 0.0